# Binary Classification of Insurance Cross Selling

#### Playground Series - Season 4, Episode 7

## Model Blend with CatBoost, LightGBM, and XGBoost. 

**Author:** Paulo Araya-Santiago

Welcome to my comprehensive notebook for the Kaggle Playground Series:Binary Classification of Insurance Cross Selling. This notebook demonstrates my end-to-end workflow, from data exploration and preprocessing to feature engineering and model tuning. The goal is to build a robust model that accurately predicts insurance responses. This particular version of this notebook is meant to be ran locally to be displayed on Github. 

## Preliminary Tests and Findings

This notebook is the culmination of the creation of at least 50 different notebooks for this competition. As of the creation of this version I still haven't been able to obtain my ideal score, but I decided to share my favorite parts of my work. I started by testing the dataset on different models with only the necessary transformations done to the features. XGBoost and LightGBM stood out from the start. I settled on trying to perfect the LightGBM model by running it through hundreds of iterations across multiple weeks on OPTUNA. When I was satisfied with the validation scores, I submitted it to the competition and the results were lackluster. This led me to take a deep dive into other notebooks scoring high in the competition (all of which are referenced below). After trying all sorts of feature engineering techniques, through testing and research I found that the dataset worked best with the addition of a few interaction features. I later found a notebook along the highest scoring ones that created a blend out of other people's submissions. I found this interesting and decided to concuct my own blend. 

## Early Explorations

I first started with a little exploratory data analysis (EDA) and basic data processing. The data comes pretty clean, but the dataset is MASSIVE. The largest I had worked with previously had been around 70k rows, but this one had 11 million. Not many columns though. Most columns were fairly easy to handle except for `Region_Code` and `Policy_Sales_Channel`. I treated those by binning them into a rare category due to the heavy imbalance towards some values. Otherwise, everything was treated pretty basically.

I made some basic EDA graphs to explore the data, using some base models to understand feature importances. Later, I discovered the magic of KLIB from another notebook: [Optuna XGBoost KLIB Notebook](https://www.kaggle.com/code/suvroo/ps4e7-optuna-xgboost-klib), which taught me a thing or two about cleaning the data easily with KLIB, and how to keep track of hyperparameter studies with Optuna and some of its also amazing graphs.

At first, I applied the basic preprocessing steps necessary to run the dataset efficiently in the models I was using. Also incorporating some downcasting to the workflow so that the model would work more efficiently. I tried applying some additional transformations to the data like removing outliers, creating rare categories for feature values with low counts, and creating KMEANS cluster feature. I also tested borrowed feature interactions from other notebooks and created my own. After possibly hundreds of tests, I settled on a combination of my own outlier removal using IQR and a set of feature interactions from [this Kaggle notebook](https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution).

## Libraries Import
In this section, we import all necessary libraries required for data manipulation, visualization, model building, and evaluation.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import gc
import klib
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, Pool
from datetime import datetime
import warnings
import joblib
import seaborn as sns


warnings.filterwarnings("ignore")

In [ ]:
# Function to create a log filename with the notebook name and current datetime
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
log_filename = f'kaggle_submission_{current_time}.log'

# Configure logging to save to a file and output to the console
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_filename),
        logging.StreamHandler()  # This ensures logs are also output to the console
    ]
)

## Data Loading
Here, we load the training dataset. This dataset will be used for all subsequent data processing and model training steps. This dataset was created artificially for the Kaggle Playground Series S4E7, based on [this set](https://www.kaggle.com/datasets/annantkumarsingh/health-insurance-cross-sell-prediction-data). For EDA, I'll use only a sample.

In [ ]:
# Paths to datasets
train_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv"
test_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\test.csv"

eda_df = pd.read_csv(train_path).sample(frac=0.005)

### Displaying Sampled Data
After performing sampling, we display the first few rows of the sampled dataset to understand its structure and verify the sampling process.

In [ ]:
eda_df.head()

In [ ]:
eda_df.info()

### Data Summary
We use the `describe` method to generate summary statistics for the numerical columns in the dataset. This provides insights into the central tendency, dispersion, and shape of the dataset’s distribution.


In [ ]:
eda_df.describe()

### Missing Values Check
It is essential to check for missing values in the dataset as they can affect the model performance. Here, we count the number of missing values in each column.



In [ ]:
eda_df.isnull().sum()

### Initial Observations and Ideas

- **Gender, Driving_License, Regional_Code, Previously_Insured, Vehicle_Age, Vehicle_Damage, Policy_Sales_Channel, and Response** are all categories. I will treat most of them as numerical columns for now, except for Gender, Vehicle_Age, Previously_Insured, and Vehicle_Damage, which I will turn into categories to use in KLIB's streamlined categorical plotting. From previous explorations, I know Driving_License only has 1 negative value, so I will drop it.
- I will later remap those four categories into numerical columns after EDA as part of the preprocessing for the model.
- I will MinMax scale Age and Vintage when standardizing because they have a reasonable range for this type of transformation.


# EDA

I turn some categories I want to plot into category dtype to be compatible with klib, they will automatically switch back to numeric during preprocessing. 

In [ ]:
# Convert specified columns to categorical
categorical_columns = ['Gender', 'Vehicle_Age', 'Previously_Insured']

for col in categorical_columns:
    eda_df[col] = eda_df[col].astype('category')

# Convert 'Previously_Insured' column to a categorical type with specific labels
eda_df['Previously_Insured'] = pd.Categorical(eda_df['Previously_Insured'], categories=[0, 1], ordered=True)
eda_df['Previously_Insured'] = eda_df['Previously_Insured'].cat.rename_categories(["Uninsured", "Insured"])


### Visualizing Data with KLIB
Using KLIB, we create categorical plots to visualize the distribution of categorical features in the dataset. This helps in understanding the balance of different categories within the dataset.

In [ ]:
klib.cat_plot(eda_df)

### KLIB Cat Plot Explanation
The KLIB categorical plot is an interesting way of visualizing binary variables within a dataset. However, it doesn't translate too well with categorical variables having more than two possible values.


### Plotting Relationships

We create a custom function to plot the relationship between categorical variables and the target in a 2x2 single figure.


In [ ]:
# Function to plot the relationship between categorical variables and the target in a 2x2 single figure
def plot_categorical_vs_target(df, cat_cols, target_col):
    num_plots = len(cat_cols)
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
    palette = ["#66c2a5", "#fc8d62"]  # Custom palette with exactly two colors

    for ax, col in zip(axes.flatten(), cat_cols):
        sns.countplot(data=df, x=col, hue=target_col, ax=ax, palette=palette)
        ax.set_title(f'Relation between {col} and {target_col}')
    
    plt.tight_layout()
    plt.show()

# Plot the relationships
categorical_columns = ['Gender', 'Vehicle_Age', 'Previously_Insured', 'Vehicle_Damage']
plot_categorical_vs_target(eda_df, categorical_columns, 'Response')


The binary categories were balanced for the most part. Gender's relation to the target doesn't tell us much knowing that there are slightly more Males to begin with. People with newer vehicles are much more likely to insure them. Naturally people that are already insured answered No (who knows if that was a no to switching providers as well). People with no vehicle damage are not very likely to ensure their cars.

### Distribution Plots with KLIB
We use KLIB to create distribution plots for several features, including Annual_Premium, Age, Region_Code, Policy_Sales_Channel, and Vintage. This helps in understanding the distribution and identifying potential outliers.


In [ ]:
# Plotting categorical features against the target variable
klib.dist_plot(eda_df[['Annual_Premium']])
klib.dist_plot(eda_df[['Age']])
klib.dist_plot(eda_df[['Region_Code']])
klib.dist_plot(eda_df[['Policy_Sales_Channel']])
klib.dist_plot(eda_df[['Vintage']])

### Distribution Plot Analysis
- **Annual_Premium** is heavily right-skewed with some heavy outliers in larger numbers. It is also bimodally distributed with a large concentration in the lower values and a second concentration near the mean. I might use a standard outlier removal method or a more aggressive quartile method. It might also benefit from log-transformation for its skewness.
- **Age** is right-skewed, but given the nature of the feature, I will leave it as is and use only a MinMax Scaler during preprocessing.
- **Region_Code** is actually a category but has many different values. It is clear certain regions are much more favored. In a previous notebook, I tried compiling all of the rarer codes into their own category. I might attempt this again but will need to consider that the rare category must be a number in itself for the model.
- **Policy_Sales_Channel** is similar to Region_Code and will be treated the same way.
- **Vintage** is the most normally distributed feature and I will probably scale it with a MinMax Scaler.

### Encoding Categorical Features

Here, we encode the 'Previously_Insured' feature. So that it works properly with the klib correlation plot.

In [ ]:
eda_df['Previously_Insured'] = eda_df['Previously_Insured'].cat.codes

### Correlation Plot with KLIB
We use KLIB to create a correlation plot to identify relationships between the features and the target variable. This helps in understanding which features might be important for the model.


In [ ]:
klib.corr_plot(eda_df, target='Response')

### Correlation Plot Analysis
The only feature that seems to have a significant relation towards conversion on the target is whether people are uninsured prior to the call. Based on this I will borrow the features that were engineered in another notebook to create categorical features that capture the relationship between other features and 'Previously_Insured'.

##### Reference: https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution

In [ ]:
del eda_df

## Custom Functions

In this section, we define custom functions to streamline our data processing and feature engineering tasks. These functions help in optimizing memory usage, handling categorical features, and generating new features from existing ones. Let's walk through each of them.

### Memory Optimization Function

Efficient memory usage is crucial when dealing with large datasets. The `reduce_mem_usage` function iterates through all columns of a dataframe and modifies their data type to reduce memory consumption. This optimization ensures that we can handle the dataset more effectively without running into memory issues.

### Safe Mapping Function

Handling categorical features often involves mapping them to numerical values. The `safe_map` function ensures that we map these categories correctly and log any unknown categories that might be encountered during the process.

### Feature Engineering Function

Feature engineering is a critical step in enhancing model performance. The `feature_engineering` function creates new features by combining existing ones. These new features capture interactions between variables that might be predictive of the target variable.


In [ ]:
# Documenting the purpose and usage of the function
def get_column_stats(df):
    """Get basic statistics for each column in the dataframe."""
    stats = {}
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            stats[col] = {
                'min': df[col].min(),
                'max': df[col].max(),
                'mean': df[col].mean(),
            }
        else:
            stats[col] = {
                'unique': df[col].nunique()
            }
    return stats

# Log comparison of statistics
def compare_stats(stats_before, stats_after):
    """Compare statistics before and after type conversion."""
    for col in stats_before:
        if stats_before[col] != stats_after[col]:
            logging.warning(f"Column {col} has changed: {stats_before[col]} != {stats_after[col]}")

# Log precision loss
def calculate_precision_loss(stats_before, stats_after):
    """Calculate and log precision loss for numeric columns."""
    for col in stats_before:
        if 'mean' in stats_before[col]:
            mean_before = stats_before[col]['mean']
            mean_after = stats_after[col]['mean']
            precision_loss = abs(mean_before - mean_after) / abs(mean_before) * 100
            logging.info(f"Column {col} precision loss: {precision_loss:.6f}%")

# Memory optimization function
def reduce_mem_usage(df, verbose=True):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    start_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logging.info(f'Start memory usage of dataframe: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        logging.info(f'End memory usage of dataframe: {end_mem:.2f} MB')
        logging.info(f'Decreased by {(100 * (start_mem - end_mem) / start_mem):.1f}%')

    return df

# Log any unknown categories during mapping
def safe_map(df, column, mapping):
    """Map categorical values to numerical values and log any unknown categories."""
    unknown_categories = set(df[column]) - set(mapping.keys())
    if unknown_categories:
        logging.warning(f"Unknown categories in column {column}: {unknown_categories}")
    df[column] = df[column].map(mapping)
    return df

# Function to import data with logging
def import_data(path, index_col=None):
    """Import data from a CSV file and optimize memory usage."""
    try:
        df = pd.read_csv(path, index_col=index_col)
        
        # Get column stats before optimization
        stats_before = get_column_stats(df)
        
        df = reduce_mem_usage(df)
        
        # Get column stats after optimization
        stats_after = get_column_stats(df)
        
        # Compare statistics and calculate precision loss
        compare_stats(stats_before, stats_after)
        calculate_precision_loss(stats_before, stats_after)
        
        logging.info(f'Data loaded and memory optimized from {path}')
        return df
    except Exception as e:
        logging.error(f'Error loading data from {path}: {str(e)}')
        return None

# Preprocess data with logging
def preprocess_data(df):
    """Preprocess the dataset."""
    gender_mapping = {'Male': 1, 'Female': 0}
    vehicle_damage_mapping = {'Yes': 1, 'No': 0}
    vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
    
    df = safe_map(df, 'Gender', gender_mapping)
    df = safe_map(df, 'Vehicle_Damage', vehicle_damage_mapping)
    df = safe_map(df, 'Vehicle_Age', vehicle_age_mapping)
    
    df.drop(['Driving_License'], axis=1, inplace=True)
    logging.info("Data preprocessing completed.")
    return df

# Feature engineering function with logging
def feature_engineering(df):
    """Feature engineering on the dataset."""
    df['Previously_Insured_Annual_Premium'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Annual_Premium'].astype(str)))[0]
    df['Previously_Insured_Vehicle_Age'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vehicle_Age'].astype(str)))[0]
    df['Previously_Insured_Vehicle_Damage'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vehicle_Damage'].astype(str)))[0]
    df['Previously_Insured_Vintage'] = pd.factorize((df['Previously_Insured'].astype(str) + df['Vintage'].astype(str)))[0]
    logging.info("Feature engineering completed.")
    return df

## Downcasting
To optimize memory usage, we downcast numerical columns to more appropriate data types without losing information. We also check preemptively the loss of precision to ensure the efficiency gains are worth it.


In [ ]:
# Load and optimize data
train_df = import_data(train_path, index_col='id')
test_df = import_data(test_path, index_col='id')

gc.collect()
logging.info("Data loaded successfully.")

## Preprocessing and Feature Engineering

To prepare our dataset for modeling, we need to preprocess it by handling missing values, encoding categorical variables, and engineering new features. This process ensures that our dataset is clean and contains relevant information for our models to learn from.

## Preprocessing
In this step, we preprocess the data by mapping the categorical features into numerical values. This prepares the dataset for model training.

In [ ]:
# Apply preprocessing
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)
logging.info("Data preprocessed successfully.")

## Feature Engineering
We create new features by capturing the interactions between them and encoding that into categorical combinations. This can provide additional information to the model and improve its performance.
##### Reference: https://www.kaggle.com/code/rohanrao/automl-grand-prix-1st-place-solution

In [ ]:
# Apply feature engineering
train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)

gc.collect()
logging.info("Feature engineering completed successfully.")

#### Feature Scaling

We apply standard scaling to the numerical features to ensure they are on a similar scale. This is important for many machine learning algorithms to perform well.

In [ ]:
# Normalize numeric columns
num_cols = ['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])
logging.info("Numeric columns normalized.")

#### Separating Features and Target Variable

We separate the features (X) and the target variable (y) from the preprocessed dataset. This is an essential step before model training.

In [ ]:
# Separate features and target variable
X = train_df.drop('Response', axis=1)
y = train_df['Response']

## Model Training and Evaluation
In this section, we train our model using LightGBM, XGBoost, and CatBoost, and evaluate their performance using cross-validation. The predictions of each fold will be averaged into that model's contribution of the blend.

## Modeling Process

For this competition, I selected CatBoost, LightGBM, and XGBoost. Each model has unique strengths, making them ideal for capturing different aspects of the data. Blending their predictions leverages these strengths for a more robust solution. Because of the massive size of the dataset I'm saving each model iteration created by each fold, in case something goes wrong during modeling. 

- **CatBoost:** Excels with categorical variables.
- **LightGBM:** Offers speed and efficiency.
- **XGBoost:** Ensures versatility and robustness.




Most of these values for hyperparameters are based on previous searches using OPTUNA or referencing other notebooks from the competition.

Reference: https://www.kaggle.com/code/darkdevil18/0-89698-ps4e7-are-you-insured

In [ ]:
# Create Stratified K-Folds
n_splits = 5
skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

## CatBoost

- **Handles Categorical Data Natively:** Reduces preprocessing needs.
- **Ordered Boosting:** Minimizes overfitting and improves generalization.

In [ ]:
# Define CatBoost parameters
cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'class_names': [0, 1],
    'learning_rate': 0.075,
    'iterations': 3000,
    'depth': 9,
    'random_strength': 0,
    'l2_leaf_reg': 0.5,
    'max_leaves': 512,
    'fold_permutation_block': 64,
    'task_type': 'GPU',  # Ensure your environment supports GPU
    'random_seed': 42,
    'allow_writing_files': False,
    'verbose': 100,  # Display log every 100 iterations
    # 'thread_count': -1
}

# Initialize lists to store out-of-fold predictions, models, and AUC scores
cat_preds = []
cat_aucs = []

test_pool = Pool(test_df.astype(str), cat_features=X.columns.values)

# CatBoost Model
logging.info("Starting CatBoost training...")
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    logging.info(f"---- CatBoost Fold {fold + 1} ----")
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[test_idx], y.iloc[test_idx]
    
    train_pool = Pool(X_train.astype(str), y_train, cat_features=X.columns.values)
    valid_pool = Pool(X_valid.astype(str), y_valid, cat_features=X.columns.values)
    
    model = CatBoostClassifier(**cat_params)
    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=50, verbose=100)
    
    valid_preds = model.predict_proba(X_valid.astype(str))[:, 1]
    auc_score = roc_auc_score(y_valid, valid_preds)
    cat_aucs.append(auc_score)
    logging.info(f"Validation AUC score for fold {fold + 1}: {auc_score:.6f}")
    
    test_pred = model.predict_proba(test_pool)[:, 1]
    cat_preds.append(test_pred)
    
    # Save the model for this fold
    joblib.dump(model, f'catboost_model_fold_{fold + 1}.pkl')
    
    # Clear memory
    del X_train, y_train, X_valid, y_valid, train_pool, valid_pool, model
    gc.collect()


# Calculate overall AUC score for CatBoost
auc_mean_cat = np.mean(cat_aucs)
auc_std_cat = np.std(cat_aucs)
logging.info(f"Overall ROC-AUC Score for CatBoost: {auc_mean_cat:.6f} ± {auc_std_cat:.6f}")

# Average the predictions from each fold for CatBoost
test_pred_cat = np.mean(cat_preds, axis=0)
joblib.dump(test_pred_cat, 'test_pred_cat.pkl')
logging.info("CatBoost models and predictions saved.")

## LightGBM

- **Histogram-Based Algorithms:** Efficient training and memory usage.
- **Leaf-Wise Growth:** Reduces loss more aggressively, enhancing accuracy.

In [ ]:
# Define LightGBM parameters
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'reg_alpha': 0.03432385172267505,
    'reg_lambda': 0.2998279059616829,
    'colsample_bytree': 0.790292183596673,
    'subsample': 0.9046878168822107,
    'learning_rate': 0.05035039561309864,
    'max_depth': 10,  # Further reduced max depth
    'num_leaves': 31,  # Standard number of leaves
    'min_child_samples': 100,  # Increased min child samples
    'min_child_weight': 1,  # Adjusted min child weight
    'min_split_gain': 0.09978597066868167,
    'max_bin': 255,
    'device': 'gpu',
    'early_stopping_rounds': 50,
    'verbose': 1  # Enable verbose mode
}

# Initialize lists to store out-of-fold predictions and AUC scores
lgb_preds = []
lgb_aucs = []

# Train LightGBM model with cross-validation
logging.info("Starting LightGBM training...")
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    logging.info(f"---- Fold {fold + 1} ----")
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[test_idx], y.iloc[test_idx]

    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid, reference=train_data)
    
    model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round=3000,
        valid_sets=[train_data, valid_data],
    )
    
    valid_preds = model.predict(X_valid, num_iteration=model.best_iteration)
    auc_score = roc_auc_score(y_valid, valid_preds)
    lgb_aucs.append(auc_score)
    logging.info(f"Validation AUC score for fold {fold + 1}: {auc_score:.6f}")
    
    test_pred = model.predict(test_df, num_iteration=model.best_iteration)
    lgb_preds.append(test_pred)
    
    # Save the model for this fold
    joblib.dump(model, f'lgb_model_fold_{fold + 1}.pkl')
    
    # Clear memory
    del X_train, y_train, X_valid, y_valid, train_data, valid_data, model
    gc.collect()

# Calculate overall AUC score for LightGBM
auc_mean_lgb = np.mean(lgb_aucs)
auc_std_lgb = np.std(lgb_aucs)
logging.info(f"Overall ROC-AUC Score for LightGBM: {auc_mean_lgb:.6f} ± {auc_std_lgb:.6f}")

# Average the predictions from each fold for LightGBM
test_pred_lgb = np.mean(lgb_preds, axis=0)
joblib.dump(test_pred_lgb, 'test_pred_lgb.pkl')
logging.info("LightGBM models and predictions saved.")

## XGBoost

- **Regularization (L1 and L2):** Prevents overfitting, improving robustness.
- **Tree Pruning:** Effectively manages overfitting.
- **Parallel Processing:** Speeds up training on large datasets.

In [ ]:
# Define XGBoost parameters
xgb_params = {
    'eval_metric': 'auc',
    'eta': 0.05,
    'alpha': 0.2545607592482198,
    'subsample': 0.8388163485383147,
    'colsample_bytree': 0.2732499701466825,
    'max_depth': 16,
    'min_child_weight': 5,
    'gamma': 0.0017688666476104672,
    'max_bin': 262143,
    'tree_method': 'gpu_hist',  # Ensure your environment supports GPU
    'predictor': 'gpu_predictor',  # Ensure your environment supports GPU
    'enable_categorical': True,
    'verbose': 100
}

# Initialize lists to store out-of-fold predictions and AUC scores
xgb_preds = []
xgb_aucs = []

# Train XGBoost model with cross-validation
logging.info("Starting XGBoost training...")
for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
    logging.info(f"---- Fold {fold + 1} ----")
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[test_idx], y.iloc[test_idx]
    
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dvalid = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)
    
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=3000,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        early_stopping_rounds=50,
        verbose_eval=100
    )
    
    valid_preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))
    auc_score = roc_auc_score(y_valid, valid_preds)
    xgb_aucs.append(auc_score)
    logging.info(f"Validation AUC score for fold {fold + 1}: {auc_score:.6f}")
    
    dtest = xgb.DMatrix(test_df, enable_categorical=True)
    test_pred = model.predict(dtest, iteration_range=(0, model.best_iteration))
    xgb_preds.append(test_pred)
    
    # Save the model for this fold
    model.save_model(f'xgb_model_fold_{fold + 1}.json')
    
    # Clear memory
    del X_train, y_train, X_valid, y_valid, dtrain, dvalid, model
    gc.collect()

# Calculate overall AUC score for XGBoost
auc_mean_xgb = np.mean(xgb_aucs)
auc_std_xgb = np.std(xgb_aucs)
logging.info(f"Overall ROC-AUC Score for XGBoost: {auc_mean_xgb:.6f} ± {auc_std_xgb:.6f}")

# Average the predictions from each fold for XGBoost
test_pred_xgb = np.mean(xgb_preds, axis=0)
joblib.dump(test_pred_xgb, 'test_pred_xgb.pkl')
logging.info("XGBoost models and predictions saved.")

## Blending Predictions
We blend predictions from CatBoost, LightGBM, and XGBoost models based on their AUC scores to create the final submission.

In [ ]:
# Normalize AUC scores to sum to 1
total_auc = auc_mean_cat + auc_mean_lgb + auc_mean_xgb

weight_cat = auc_mean_cat / total_auc
weight_lgb = auc_mean_lgb / total_auc
weight_xgb = auc_mean_xgb / total_auc

# Print weights for verification
print(f"Weights - CatBoost: {weight_cat:.4f}, LightGBM: {weight_lgb:.4f}, XGBoost: {weight_xgb:.4f}")

# Blending predictions with calculated weights
blended_preds = (weight_cat * test_pred_cat + weight_lgb * test_pred_lgb + weight_xgb * test_pred_xgb)

# Create the submission DataFrame
submission = pd.DataFrame({
    'id': test_df.index,
    'Response': blended_preds
})

# Save the submission file
submission.to_csv("submission.csv", index=False)

print("Submission file created successfully!")

## What We Did in This Notebook

This notebook showcased a detailed and comprehensive end-to-end workflow for tackling the Kaggle Playground Series S4E07 competition. Here’s a recap of our journey:

### Exploratory Data Analysis (EDA)
- **Initial Explorations:** We kicked off with a subset of the data to get a quick grasp of its structure and nuances. This initial dive helped us spot potential issues and interesting patterns early on.
- **KLIB Visualizations:** We utilized KLIB for streamlined visualizations and correlation analysis, making it easier to pinpoint key features and understand their relationships with the target variable.

### Data Preprocessing
- **Handling Missing Values:** We meticulously checked for missing values to ensure data integrity throughout the modeling process.
- **Categorical Encoding:** By converting categorical variables into numerical formats, we prepped the data for machine learning models.
- **Memory Optimization:** To efficiently handle the large dataset, we optimized memory usage through downcasting of data types.

### Feature Engineering
- **Creating New Features:** We got creative with feature engineering, developing new features by combining existing ones. This allowed us to capture interactions between variables that could boost model performance.
- **Scaling Numerical Features:** Standard scaling was applied to numerical features to bring them onto a similar scale, which is essential for many machine learning algorithms to perform optimally.

### Modeling and Evaluation
- **Model Selection:** We chose three powerhouse models – CatBoost, LightGBM, and XGBoost – each known for its unique strengths in handling structured data.
- **Cross-Validation:** To get a reliable estimate of model performance, we used stratified k-fold cross-validation, ensuring our models were tested across diverse subsets of the data.
- **Hyperparameter Tuning:** Leveraging Optuna, we efficiently tuned the hyperparameters, squeezing out the best performance from our models.

### Blending Predictions
- **Ensemble Approach:** Instead of relying on a single model, we blended predictions from CatBoost, LightGBM, and XGBoost based on their ROC-AUC scores. This ensemble method harnessed the strengths of each model, resulting in superior overall performance.

### Conclusion

In this notebook, we performed exploratory data analysis, preprocessing, feature engineering, and model training with blending at the end to form our best submission possible.. The final submission was blended using CatBoost, LightGBM and XGBoost with the best hyperparameters found by Optuna(or borrowed from other noteboks) and evaluated on the validation set. The results demonstrate the effectiveness of the selected features and the tuned model. This notebook was designed to showcase a clean and optimized workflow.

## Challenges and Solutions

- **Handling Large Datasets:** The 11 million rows were daunting. I downsampled the dataset for initial exploration to manage computational resources better.
- **Imbalanced Dataset:** SMote was my go-to solution for balancing the target variable classes. SMote resulted in too much overfitting.
- **Hyperparameter Tuning:** Given the computational constraints, I leaned heavily on Optuna for efficient hyperparameter tuning. Hyperparameter tuning took almost two weeks without many gains.

## References

Walter Reade, & Ashley Chow. (2024). *Binary Classification of Insurance Cross Selling*. Kaggle. Retrieved from [https://kaggle.com/competitions/playground-series-s4e7](https://kaggle.com/competitions/playground-series-s4e7)
